# Testing file 
### where we evaluate Zhang's models using the test set

## Preliminaries

In [1]:
from math import sqrt, isnan
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.data import Dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

from util import metrics
from util.load_data import load_data
from util.evaluation import *

from fairgan.models import *
from fairgan.learning import *

In [2]:
batch_size = 64
epochs = 100
lr = 0.001
opt = Adam(learning_rate=lr)

In [3]:
test_loop = 1

## Load data

In [4]:
x_train, y_train, a_train = load_data('adult', 'train')
raw_data = (x_train, y_train, a_train)

In [5]:
xdim = x_train.shape[1]
ydim = y_train.shape[1]
adim = a_train.shape[1]
zdim = 8

In [6]:
train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
train_data = train_data.batch(batch_size, drop_remainder=True)
train_data

<BatchDataset shapes: ((64, 113), (64, 1), (64, 1)), types: (tf.float64, tf.float64, tf.float64)>

In [7]:
x_valid, y_valid, a_valid = load_data('adult', 'valid')

valid_data = Dataset.from_tensor_slices((x_valid, y_valid, a_valid))
valid_data = valid_data.batch(batch_size, drop_remainder=True)

In [8]:
x_test, y_test, a_test = load_data('adult', 'test')

test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

In [9]:
num_batchs = y_test.shape[0]//batch_size
num_batchs *= batch_size
y_test = y_test[:num_batchs]
a_test = a_test[:num_batchs]

## Result file

In [10]:
header = "model_name", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp", "TN_a0", "FP_a0", "FN_a0", "TP_a0", "TN_a1", "FP_a1", "FN_a1", "TP_a1"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### Zhang for DP

In [11]:
fairdef = 'DemPar'

for i in range(test_loop):
    opt = Adam(learning_rate=lr)
    model = FairLogisticRegression(xdim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs, opt)

    Y_hat, A_hat = evaluation(model, test_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(y_test, Y_hat, a_test, A_hat)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    result = ['Zhang4DP', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

    results.append(result)

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.41985824704170227 | 0.7958990931510925 | 0.7547662466843501 | 0.3273375331564987
> 2 | 1.0354886054992676 | 0.6687623262405396 | 0.8144479442970822 | 0.3868534482758621
> 3 | 1.0318939685821533 | 0.6110285520553589 | 0.8206647877984085 | 0.6441064323607427
> 4 | 0.6960029006004333 | 0.5837156772613525 | 0.8297413793103449 | 0.7141495358090186
> 5 | 0.9805105328559875 | 0.5663491487503052 | 0.8360825596816976 | 0.7120772546419099
> 6 | 0.3829643130302429 | 0.5573411583900452 | 0.8416777188328912 | 0.6932609416445623
> 7 | 0.3636026978492737 | 0.5365790128707886 | 0.8407659151193634 | 0.684888925729443
> 8 | 0.7166281938552856 | 0.5501534938812256 | 0.8445789124668435 | 0.6815318302387268
> 9 | 1.0129873752593994 | 0.5469249486923218 | 0.8423822944297082 | 0.6807858090185677
> 10 | 1.0095266103744507 | 0.54656583070755 | 0.8320208885941645 | 0.6812002652519894
> 11 | 1.0033175945281982 | 0.5453625321388245 | 0.8367871352785146

### Zhang for Eq Odds

In [12]:
fairdef = 'EqOdds'

for i in range(test_loop):
    opt = Adam(learning_rate=lr)
    model = FairLogisticRegression(xdim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs, opt)

    Y_hat, A_hat = evaluation(model, test_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(y_test, Y_hat, a_test, A_hat)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    result = ['Zhang4EqOdds', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

    results.append(result)

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.8880362510681152 | 0.7379366159439087 | 0.7536057692307692 | 0.3326840185676393
> 2 | 0.3920368552207947 | 0.595535397529602 | 0.8192970822281167 | 0.5264008620689655
> 3 | 1.030700445175171 | 0.5585260987281799 | 0.8283322281167109 | 0.7106266578249336
> 4 | 1.032625675201416 | 0.5441094040870667 | 0.8251823607427056 | 0.7202834880636605
> 5 | 0.4029202163219452 | 0.5363216400146484 | 0.8282907824933687 | 0.7244694960212201
> 6 | 0.3809119462966919 | 0.5224566459655762 | 0.8337201591511936 | 0.7265417771883289
> 7 | 0.7461276054382324 | 0.5393401384353638 | 0.8391909814323607 | 0.724883952254642
> 8 | 0.3661111295223236 | 0.5114368796348572 | 0.8455736074270557 | 0.7207393899204244
> 9 | 0.35401415824890137 | 0.5024999380111694 | 0.8430039787798409 | 0.7177138594164456
> 10 | 0.36830073595046997 | 0.5135820508003235 | 0.8476873342175066 | 0.7158073607427056
> 11 | 0.3678044080734253 | 0.5143375396728516 | 0.8447446949602122

### Zhang for Eq Opp

In [13]:
fairdef = 'EqOpp'

for i in range(test_loop):
    opt = Adam(learning_rate=lr)
    model = FairLogisticRegression(xdim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs, opt)

    Y_hat, A_hat = evaluation(model, test_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(y_test, Y_hat, a_test, A_hat)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    result = ['Zhang4EqOpp', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

    results.append(result)

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.4192975163459778 | 0.2229214310646057 | 0.7571286472148542 | 0.32949270557029176
> 2 | 0.3927159011363983 | 0.16524718701839447 | 0.8105106100795756 | 0.42100464190981435
> 3 | 0.4386526942253113 | 0.12488310039043427 | 0.8232344164456233 | 0.5319545755968169
> 4 | 1.0402820110321045 | 0.11261849105358124 | 0.8345905172413793 | 0.596816976127321
> 5 | 1.0342415571212769 | 0.10793241858482361 | 0.8287881299734748 | 0.6348640583554377
> 6 | 0.38009536266326904 | 0.11873888969421387 | 0.8367042440318302 | 0.648914124668435
> 7 | 0.3784114420413971 | 0.11871960014104843 | 0.8429210875331565 | 0.6554625331564987
> 8 | 1.0152990818023682 | 0.10325539112091064 | 0.8420092838196287 | 0.6622181697612732
> 9 | 0.9706503748893738 | 0.10375283658504486 | 0.8322695623342176 | 0.6661140583554377
> 10 | 0.5435814261436462 | 0.10506999492645264 | 0.8389837533156499 | 0.6591097480106101
> 11 | 0.3388862609863281 | 0.10621694475412369 | 0.847

## Saving into DF then CSV

In [14]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp,TN_a0,FP_a0,FN_a0,TP_a0,TN_a1,FP_a1,FN_a1,TP_a1
0,Zhang4DP,0.829122,0.816432,0.913289,0.917105,0.822728,0.869173,0.870897,4182.0,168.0,265.0,292.0,6089.0,903.0,1234.0,1907.0
1,Zhang4EqOdds,0.831516,0.814250,0.909391,0.909809,0.822793,0.868712,0.868903,4196.0,154.0,268.0,289.0,6108.0,884.0,1228.0,1913.0
2,Zhang4EqOpp,0.835904,0.812956,0.901689,0.893156,0.824270,0.867551,0.863582,4231.0,119.0,281.0,276.0,6173.0,819.0,1249.0,1892.0


In [16]:
#result_df.to_csv('results/test_fairgan-100.csv')